In [4]:
#market-price

import requests
import pandas as pd
import  numpy as np
import time
from datetime import datetime

start_date = datetime(2020, 1, 1)
end_date = datetime(2020, 12, 31)

# 转换为UNIX时间戳（秒）
start_time = int(start_date.timestamp()) * 1000  # 转换为毫秒
end_time = int(end_date.timestamp()) * 1000  # 转换为毫秒

# 设置Cybotrade API的URL，使用 start_time 和 end_time，并且使用 15m 的时间间隔
url = f'https://api.datasource.cybotrade.rs/bybit-linear/candle?symbol=BTCUSDT&interval=4h&start_time={start_time}&end_time={end_time}'

# 设置请求头
headers = {
    "X-API-KEY": "9nvHJT2ud8KuQZs4s3bsIv4FiKs1GMoVReVLxhCwevI0v7uV"  # 替换为您的Cybotrade API密钥
}

# 发送请求
response = requests.get(url, headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    raw_data = response.json()['data']
    if raw_data:
        df = pd.DataFrame(raw_data, columns=['start_time', 'open', 'high', 'low', 'close', 'volume'])
        df['start_time'] = pd.to_datetime(df['start_time'], unit='ms')
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        df.set_index('start_time', inplace=True)

        # === 波动率（Volatility）===
        df['log_return'] = np.log(df['close'] / df['close'].shift(1))
        df['volatility_24h'] = df['log_return'].rolling(window=6).std()

        # === RSI===
        delta = df['close'].diff()

        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))

        # === 保存 CSV ===
        df.to_csv('bitcoin_2020.csv')
        print("✅ 数据保存成功：crypto_data_features.csv")
        print(df.tail(10))  # 打印最后 10 行数据查看结果
    else:
        print('❌ API 返回的数据为空')
else:
    print(f"❌ 请求失败，状态码：{response.status_code}")


✅ 数据保存成功：crypto_data_features.csv
                        open     high      low    close    volume  log_return  \
start_time                                                                      
2020-12-29 00:00:00  27087.5  27113.5  26473.5  26479.5  2369.966   -0.022702   
2020-12-29 04:00:00  26479.5  26610.5  25930.0  26414.0  4433.094   -0.002477   
2020-12-29 08:00:00  26414.0  26914.0  26414.0  26886.0  3971.588    0.017712   
2020-12-29 12:00:00  26886.0  27108.0  26480.0  26633.5  3619.643   -0.009436   
2020-12-29 16:00:00  26633.5  26956.0  26272.5  26937.0  2724.253    0.011331   
2020-12-29 20:00:00  26937.0  27427.5  26808.5  27401.0  3477.645    0.017079   
2020-12-30 00:00:00  27401.0  28280.0  27401.0  28094.0  3941.271    0.024977   
2020-12-30 04:00:00  28094.0  28633.0  27764.0  28431.0  4760.974    0.011924   
2020-12-30 08:00:00  28431.0  28615.5  27397.0  27877.0  6425.955   -0.019678   
2020-12-30 12:00:00  27877.0  28336.0  27621.0  28171.5  3354.710    0.0105

In [143]:
#on-chain data

import requests
import pandas as pd
import time
from datetime import datetime

start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)

# 转换为UNIX时间戳（秒）
start_time = int(start_date.timestamp()) * 1000  # 转换为毫秒
end_time = int(end_date.timestamp()) * 1000  # 转换为毫秒

# 设置Cybotrade API的URL，使用 start_time 和 end_time（不使用 limit）
# 构建获取2022年链上活跃度数据的URL
url = f"https://api.datasource.cybotrade.rs/cryptoquant/btc/exchange-flows/transactions-count?exchange=binance&window=hour&start_time={start_time}&end_time={end_time}"

# 设置请求头
headers = {
    "X-API-KEY": "9nvHJT2ud8KuQZs4s3bsIv4FiKs1GMoVReVLxhCwevI0v7uV"  # 替换为您的Cybotrade API密钥
}

# 发送请求
response = requests.get(url, headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    raw_data = response.json()['data']
    if raw_data:
        df = pd.DataFrame(raw_data, columns=['datetime', 'transactions_count_inflow', 'transactions_count_outflow'])
        df[['transactions_count_inflow', 'transactions_count_outflow']] = df[['transactions_count_inflow', 'transactions_count_outflow']].astype(float)
        df['datetime'] = pd.to_datetime(df['datetime'])
        df.set_index('datetime', inplace=True)

        #净交易量
        transactions_count = df['transactions_count_inflow'] + df['transactions_count_outflow']
        df['total_transactions_count'] = transactions_count

        df_resampled = df.resample('4H').sum()

        # === 保存 CSV ===
        df_resampled.to_csv('bitcoun_transactions_count_2022.csv')
        print("✅ 数据保存成功")
        print(df.tail(10))  # 打印最后 10 行数据查看结果
    else:
        print('❌ API 返回的数据为空')
else:
    print(f"❌ 请求失败，状态码：{response.status_code}")


✅ 数据保存成功
                     transactions_count_inflow  transactions_count_outflow  \
datetime                                                                     
2022-12-30 06:00:00                      710.0                        60.0   
2022-12-30 07:00:00                     1028.0                        26.0   
2022-12-30 08:00:00                     1582.0                        65.0   
2022-12-30 09:00:00                      704.0                        19.0   
2022-12-30 10:00:00                     1634.0                        83.0   
2022-12-30 11:00:00                      688.0                        22.0   
2022-12-30 12:00:00                     2037.0                        78.0   
2022-12-30 13:00:00                      861.0                        21.0   
2022-12-30 14:00:00                     2154.0                        89.0   
2022-12-30 15:00:00                     1866.0                        34.0   

                     total_transactions_count  
dateti

C:\Users\lisha\AppData\Local\Temp\ipykernel_19824\1641050072.py:38: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('4H').sum()


In [174]:
#whale ratio
import requests
import pandas as pd
import time
from datetime import datetime

start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)

# 转换为UNIX时间戳（秒）
start_time = int(start_date.timestamp()) * 1000  # 转换为毫秒
end_time = int(end_date.timestamp()) * 1000  # 转换为毫秒

# 设置Cybotrade API的URL，使用 start_time 和 end_time（不使用 limit）
# 构建获取2022年链上活跃度数据的URL
url = f"https://api.datasource.cybotrade.rs/cryptoquant/btc/flow-indicator/exchange-whale-ratio?exchange=binance&window=hour&start_time={start_time}&end_time={end_time}"

# 设置请求头
headers = {
    "X-API-KEY": "9nvHJT2ud8KuQZs4s3bsIv4FiKs1GMoVReVLxhCwevI0v7uV"  # 替换为您的Cybotrade API密钥
}

# 发送请求
response = requests.get(url, headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    raw_data = response.json()['data']
    if raw_data:
        df = pd.DataFrame(raw_data, columns=['datetime', 'exchange_whale_ratio'])
        df[['exchange_whale_ratio']] = df[['exchange_whale_ratio']].astype(float)
        df['datetime'] = pd.to_datetime(df['datetime'])
        df.set_index('datetime', inplace=True)

        #大额交易占比
        df_resampled = df.resample('4H').mean()
        stats = df['exchange_whale_ratio'].describe()
        print(stats)

        # === 保存 CSV ===
        df_resampled.to_csv('bitcoun_exchange_whale_ratio_2022.csv')
        print("✅ 数据保存成功")
        print(df.tail(10))  # 打印最后 10 行数据查看结果
    else:
        print('❌ API 返回的数据为空')
else:
    print(f"❌ 请求失败，状态码：{response.status_code}")


count    8736.000000
mean        0.675960
std         0.146107
min         0.000000
25%         0.578781
50%         0.680896
75%         0.780558
max         0.998240
Name: exchange_whale_ratio, dtype: float64
✅ 数据保存成功
                     exchange_whale_ratio
datetime                                 
2022-12-30 06:00:00              0.664910
2022-12-30 07:00:00              0.591480
2022-12-30 08:00:00              0.582495
2022-12-30 09:00:00              0.514587
2022-12-30 10:00:00              0.682205
2022-12-30 11:00:00              0.834181
2022-12-30 12:00:00              0.737111
2022-12-30 13:00:00              0.707211
2022-12-30 14:00:00              0.614143
2022-12-30 15:00:00              0.815125


C:\Users\lisha\AppData\Local\Temp\ipykernel_19824\1388739442.py:35: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('4H').mean()


In [187]:
#exchange-netflow

import requests
import pandas as pd
import time
from datetime import datetime

start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)

# 转换为UNIX时间戳（秒）
start_time = int(start_date.timestamp()) * 1000  # 转换为毫秒
end_time = int(end_date.timestamp()) * 1000  # 转换为毫秒

# 设置Cybotrade API的URL，使用 start_time 和 end_time（不使用 limit）
# 构建获取2022年链上活跃度数据的URL
url = f"https://api.datasource.cybotrade.rs/cryptoquant/btc/exchange-flows/netflow?exchange=binance&window=hour&start_time={start_time}&end_time={end_time}"

# 设置请求头
headers = {
    "X-API-KEY": "9nvHJT2ud8KuQZs4s3bsIv4FiKs1GMoVReVLxhCwevI0v7uV"  # 替换为您的Cybotrade API密钥
}

# 发送请求
response = requests.get(url, headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    raw_data = response.json()['data']
    if raw_data:
        df = pd.DataFrame(raw_data, columns=['datetime', 'netflow_total'])
        df[['netflow_total']] = df[['netflow_total']].astype(float)
        df['datetime'] = pd.to_datetime(df['datetime'])
        df.set_index('datetime', inplace=True)

        #大额交易占比
        df.rename(columns={'netflow_total': 'exchange_netflow'}, inplace=True)
        df_resampled = df.resample('4H').mean()

        # === 保存 CSV ===
        df_resampled.to_csv('bitcoun_cryptoquant_exchange_netflow_2022.csv')
        print("✅ 数据保存成功")
        print(df.tail(10))  # 打印最后 10 行数据查看结果
    else:
        print('❌ API 返回的数据为空')
else:
    print(f"❌ 请求失败，状态码：{response.status_code}")


✅ 数据保存成功
                     exchange_netflow
datetime                             
2022-12-30 06:00:00         -0.772888
2022-12-30 07:00:00        -81.898316
2022-12-30 08:00:00        276.473049
2022-12-30 09:00:00         79.400314
2022-12-30 10:00:00        602.002721
2022-12-30 11:00:00        -10.417641
2022-12-30 12:00:00        868.962610
2022-12-30 13:00:00         48.039420
2022-12-30 14:00:00        289.344773
2022-12-30 15:00:00      -1281.322299


C:\Users\lisha\AppData\Local\Temp\ipykernel_19824\347900193.py:36: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('4H').mean()
